In [91]:
%pylab inline
from __future__ import division, print_function, absolute_import

import tflearn
import numpy as np
from pprint import pprint
from PIL import Image
from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python2.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['shuffle']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


### Download CIFAR-10 and Prepare X, and Y dataset

| Variable | Shape   | Description |
|:---------|:--------|:------------|
|   X      | (50000, 32, 32, 3)    | 32 * 32 (RGB 3개) 이미지들 50000개의 데이터셋 |
|   Y      | (50000, 10)           | [0, 0, 0, 1, 0, 0, 0, 0, 0, 0] 처럼 정답이 들어 있는 50000개의 데이터셋 |
| X_test   | (10000, 32, 32, 3)    | 10000개의 테스트 데이터셋 |
| Y_test   | (10000, 10)           | 10000개의 정답이 들어있는 테스트 데이터 셋 |


**to_categorical(Y, 10)** <br>
원래 Y의 값은 1 dimentional array (50000,) 로서 5만개의 정답을 갖고 있는 데이터셋. <br>
하지만 to_categorical 함수를 거치면 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0] 처럼 머신러닝에서 사용가능한 패턴으로 변화됨


In [15]:
# Data loading and preprocessing
from tflearn.datasets import cifar10
(X, Y), (X_test, Y_test) = cifar10.load_data()
X, Y = shuffle(X, Y)
Y = to_categorical(Y, 10)
Y_test = to_categorical(Y_test, 10)

In [22]:
# Real-time data preprocessing
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()


In [23]:
# Real-time data augmentation
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)


In [24]:
# Convolutional network building
network = input_data(shape=[None, 32, 32, 3],
                     data_preprocessing=img_prep,
                     data_augmentation=img_aug)
network = conv_2d(network, 32, 3, activation='relu')
network = max_pool_2d(network, 2)
network = conv_2d(network, 64, 3, activation='relu')
network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network, 2)
network = fully_connected(network, 512, activation='relu')
network = dropout(network, 0.5)
network = fully_connected(network, 10, activation='softmax')
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)


In [25]:
# Train using classifier
model = tflearn.DNN(network, tensorboard_verbose=0)
model.fit(X, Y, n_epoch=50, shuffle=True, validation_set=(X_test, Y_test),
          show_metric=True, batch_size=96, run_id='cifar10_cnn')

Training Step: 26050  | total loss: 0.37202
| Adam | epoch: 050 | loss: 0.37202 - acc: 0.8723 | val_loss: 0.67760 - val_acc: 0.8154 -- iter: 50000/50000
Training Step: 26050  | total loss: 0.37202
| Adam | epoch: 050 | loss: 0.37202 - acc: 0.8723 | val_loss: 0.67760 - val_acc: 0.8154 -- iter: 50000/50000
--


### TensorBoard

아래의 코드를 통해서 Graph를 볼 수 있습니다.

```
tensorboard --logdir='/tmp/tflearn_logs'
```


In [159]:
print('Predict')
pprint(model.predict(X[100:101].copy()))

print('\nAnswer')
pprint(Y[100:101].tolist())


Predict
[[4.0301410731330883e-17,
  1.984033701774024e-07,
  1.308805266897497e-16,
  3.92242015437439e-18,
  5.195401066204092e-22,
  1.0799233868966178e-20,
  4.716609702171506e-22,
  4.3049110545583245e-19,
  1.0658509665140148e-12,
  0.9999997615814209]]

Answer
[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]]


<tf.Tensor 'FullyConnected_1/Softmax:0' shape=(?, 10) dtype=float32>